## **Utrecht Leek Trips**

In [1]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

### Extracting Common Platedates

- In this step we'll extract the `platedate` that passed through `area_list = ['start_location','end_location']` on the same day.

- We'll save these values as a list and use it later to extract trips.

In [2]:
##TODO -> Create functions for repetitive tasks

# ,skiprows=range(3, 260000000)
# Input the csv
# Extracting a subset of 1000000 rows by default
def load_data(dir_name,base_filename):
    complete_path=os.path.join(dir_name, base_filename + "." + "csv")
    # df=pd.read_csv(complete_path,sep=';',encoding= 'unicode_escape',nrows=10000,engine='c',infer_datetime_format=True,usecols=['Trip_Summary_Id','Numberplate','Point_time-stamp','Latitude',
    #                                                                                                                           'Longitude','Meters_travelled','Time_elapsed','Point_speed','Number_of_lanes',
    #                                                                                                                           'Road_type','Road_class','Road_form','Speed_restriction','TNO_Time-stamp'])
    # df = pd.concat((chunk for chunk in pd.read_csv(complete_path,sep=';',nrows=5000000,encoding= 'unicode_escape',engine='c',infer_datetime_format=True,chunksize=1000000,low_memory=False,usecols=['Numberplate','Latitude',
    #                                                                                                                           'Longitude','Meters_travelled','Time_elapsed','Point_speed','Number_of_lanes',
    #                                                                                                                           'Road_type','Road_class','Speed_restriction','TNO_Time-stamp'])))
    
    df = pd.concat((chunk for chunk in pd.read_csv(complete_path,sep=';',nrows=20000000,encoding= 'unicode_escape',engine='c',infer_datetime_format=True,chunksize=1000000,low_memory=False)))
    return df

# Dropping the first and last row of csv ("------")
def drop_first_row(df):
    df=df.iloc[1:]
    df=df[:-1]
    return df

def resetIndex(df):
    return df.reset_index()


##TODO -> Rewrite this function

def cast_to_correct_dtype(df):
    
    if 'Id' in df.columns:
        df['Id'] = df['Id'].astype('int')
        
    if 'AOS_position_Id' in df.columns:
        df['AOS_position_Id'] = df['AOS_position_Id'].astype('int')
    
    if 'Acceleration_x' in df.columns:
        df['Acceleration_x'] = df['Acceleration_x'].astype('float')
        
    if 'Acceleration_y' in df.columns:
        df['Acceleration_y'] = df['Acceleration_y'].astype('float')
        
    if 'TNO_Valid' in df.columns:
        df['TNO_Valid'] = df['TNO_Valid'].astype('int')
    
    if 'Latitude' in df.columns:
        df['Latitude'] = df['Latitude'].astype('float')
        
    if 'Longitude' in df.columns:
        df['Longitude'] = df['Longitude'].astype('float')
        
    if 'Event/action_speed' in df.columns:
        df['Event/action_speed'] = df['Event/action_speed'].astype('int')
        
    if 'Event/action_type' in df.columns:
        df['Event/action_type'] = df['Event/action_type'].astype('int')    
        
    if 'Number_of_lanes' in df.columns:
        df['Number_of_lanes'] = df['Number_of_lanes'].astype('int')
        
    if 'Road_class' in df.columns:
        df['Road_class'] = df['Road_class'].astype('int')
        
    if 'Road_type' in df.columns:
        df['Road_type'] = df['Road_type'].astype('int')
        
    if 'Crash_position_Id' in df.columns:
        df['Crash_position_Id']=df['Crash_position_Id'].astype('int')
        
    if 'Crash_position_Id' in df.columns:
        df['Crash_position_Id'] = df['Crash_position_Id'].astype('int')
        
    if 'Point_speed' in df.columns:
        df['Point_speed'] = df['Point_speed'].astype('float')
        
    if 'Average_speed_fpp' in df.columns:
        df['Average_speed_fpp'] = df['Average_speed_fpp'].astype('float')  
        
    if 'Average_Speed' in df.columns:
        df['Average_Speed'] = df['Average_Speed'].astype('float')        

    if 'Maximum_speed' in df.columns:
        df['Maximum_speed'] = df['Maximum_speed'].astype('float')     
        
    if 'Meters_travelled' in df.columns:
        df['Meters_travelled'] = df['Meters_travelled'].astype('int')  

    if 'Road_form' in df.columns:
        df['Road_form'] = df['Road_form'].astype('int') 
        
    if 'Speed_restriction' in df.columns:
        df['Speed_restriction'] = df['Speed_restriction'].astype('int') 
        
    if 'Crash_speed' in df.columns:
        df['Crash_speed'] = df['Crash_speed'].astype('int')
        
    if 'Maximum_acceleration' in df.columns:
        df['Maximum_acceleration'] = df['Maximum_acceleration'].astype('float')
        
    if 'Numberplate' in df.columns:
        df['Numberplate']=df['Numberplate'].astype('str')
        

    return df

            

## Handling date time related fields

def cast_date_time(df):
    if 'AOS_event/action_time' in df.columns:
        df['AOS_event/action_time']=pd.datetime(df['AOS_event/action_time'],errors='coerce')
        df['AOS_event/action_time_hour']=df['AOS_event/action_time'].dt.hour
    
    if 'TNO_Trip-start' in df.columns:
        df['TNO_Trip-start'] = pd.to_datetime(df['TNO_Trip-start'],errors='coerce')
        df['TNO_Trip-start_hour'] = df['TNO_Trip-start'].dt.hour
        
    if 'TNO_Trip-end' in df.columns:
        df['TNO_Trip-end'] = pd.to_datetime(df['TNO_Trip-end'],errors='coerce')
        
    if 'Position_time' in df.columns:
        df['Position_time'] = pd.to_datetime(df['Position_time'],errors='coerce')

    if 'TNO_Time-stamp' in df.columns:
        df['TNO_Time-stamp'] = pd.to_datetime(df['TNO_Time-stamp'],errors='coerce')
        df['TNO_Time-stamp_hour'] = df['TNO_Time-stamp'].dt.hour
        
    return df

## Converting Time Stamps to datetime

def date_and_time_columns(df):
    if 'AOS_event/action_time' in df.columns:
        df['AOS Trip Date']=df['AOS_event/action_time'].dt.date
        df['AOS Event Time']=df['AOS_event/action_time'].dt.time
        df['AOS Trip Hour']=df['AOS_event/action_time'].dt.hour
    
    
    if 'TNO_Time-stamp' in df.columns:
        df['TNO Trip Date'] = df['TNO_Time-stamp'].dt.date
        df['TNO Trip Time'] = df['TNO_Time-stamp'].dt.time
        df['TNO Trip Hour']=df['TNO_Time-stamp'].dt.hour
        # df['TNO Trip Hour'] = df['TNO Trip Time'].dt.hour
    return df

def Time_of_the_day(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x < 12 ):
        return 'Morning'
    elif (x >= 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Evening'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    
    
def rename_some_stuff(df):
    if 'Event/action_type' in df.columns:
        df["Event type Rename"]=df["Event/action_type"].replace({0: 'Headway Warning = OFF', 1: 'Headway Warning (long)', 2: 'Headway Warning (medium)'
                                                                                 ,3: 'Headway Warning (short)',10:'Lane Departure Warning = OFF',11:'Left Lane Departure Warning = ON'
                                                                                 ,12:'Right Lane Departure Warning = ON',13:'Left and Right Lane Departure Warning = ON'
                                                                                 ,20:' Indicators = OFF',21:'Left Indicator = ON',22:'Right Indicator = ON',23:'Left and Right Indicator = ON'
                                                                                 ,40:'Brakes = OFF',41:'Brakes = ON'})
        
    if 'Road_type' in df.columns:
        df["Road_type_Rename"]=df["Road_type"].replace({0: "Urban", 1: "Motorway",2:"Extra Urban",3:"Unavailable"})
        
    return df

def detect_overspeeding_count(df):
    # create a list of our conditions
    # if 'Event/action_speed' and 'Speed_restriction' in df.columns:
    #     conditions = [(df['Event/action_speed']> df['Speed_restriction'])]
    #     # # create a list of the values we want to assign for each condition
    #     values = [1]
    #     # create a new column and use np.select to assign values to it using our lists as arguments
    #     df['Overspeeding_event'] = np.select(conditions, values)
        
    # return df

    if 'Point_speed' and 'Speed_restriction' in df.columns:
        conditions = [(df['Point_speed']> df['Speed_restriction'])]
        # # create a list of the values we want to assign for each condition
        values = [1]
        # create a new column and use np.select to assign values to it using our lists as arguments
        df['Overspeeding_event'] = np.select(conditions, values)
        
    return df

def some_processing(df):
    if 'Meters_travelled' in df.columns:
        df['KmTravelled']=df['Meters_travelled']/1000
    return df

In [3]:
location1='Rotterdam' # Define Location 1 here
location2='Amsterdam' # Define Location 2 here

### Trip Detail

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar



area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape', usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area'])

df = df[df['Admin_area'].isin(area_list)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

### AOS Summary

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar



area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', encoding='unicode_escape', usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area'])

df = df[df['Admin_area'].isin(area_list)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
## Casting `TNO_Time-stamp` to correct dtype
df['TNO_Time-stamp'] = pd.to_datetime(df['TNO_Time-stamp'],errors='coerce')
def date_and_time_columns(df):
    if 'TNO_Time-stamp' in df.columns:
        df['TNO Trip Date'] = df['TNO_Time-stamp'].dt.date
    return df

df=date_and_time_columns(df)
df['TNO_date_str'] = df['TNO Trip Date'].apply(lambda x: str(x))

In [ ]:
df_location1=df[(df['Admin_area']==location1)] # We can replace these locations to anything else
df_location2=df[(df['Admin_area']==location2)]

df_location1=cast_to_correct_dtype(df_location1)
df_location2=cast_to_correct_dtype(df_location2)

In [ ]:
common_utrecht_leek_numberplate=(np.intersect1d(df_location1['Numberplate'], df_location2['Numberplate']))
common_utrecht_leek_platedate=(np.intersect1d(df_location1['platedate'], df_location2['platedate']))
# common_utrecht_leek_numberplate

In [ ]:
# Saving common platedate list as pkl file

with open("common_utrecht_leek_numberplate_aos_summary", "wb") as fp:   #Pickling
    pickle.dump(common_utrecht_leek_numberplate, fp)
    
with open("common_utrecht_leek_platedate_aos_summary", "wb") as fp:   #Pickling
    pickle.dump(common_utrecht_leek_platedate, fp)

In [ ]:
df_utrecht_points = df_location1[df_location1['Numberplate'].isin(common_utrecht_leek_numberplate)]
df_leek_points = df_location2[df_location2['Numberplate'].isin(common_utrecht_leek_numberplate)]
# df_utrecht_points

## **Part - III**

In [4]:
#### Common platedate - loading pkl file
with open("common_rotterdam_amsterdam_platedate", "rb") as fp:
    common_utrecht_leek_platedate = pickle.load(fp)
    
with open("common_rotterdam_amsterdam_numberplate", "rb") as fp:
    common_utrecht_leek_numberplate = pickle.load(fp)

In [ ]:
len(common_utrecht_leek_platedate[501:1000])

In [ ]:
len(common_utrecht_leek_platedate)

Trip Detail

In [5]:
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_20M.csv',assume_missing=True,on_bad_lines='skip') # AOS SUMMARY

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',assume_missing=True,on_bad_lines='skip') # TRIP DETAIL


## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 
df = df[df['platedate'].isin(common_utrecht_leek_platedate)]
with ProgressBar():
    df_trip_utrecht_leek = df.compute()
    
df_trip_utrecht_leek['Numberplate_copy'] = df_trip_utrecht_leek['Numberplate']
df_trip_utrecht_leek['platedate_copy'] = df_trip_utrecht_leek['platedate']
# len(df_trip_utrecht_leek['platedate_copy'].unique())

[########################################] | 100% Completed |  1min 35.6s


AOS Summary

In [6]:
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_20M.csv',assume_missing=True,on_bad_lines='skip') # AOS SUMMARY

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_20M.csv',assume_missing=True,on_bad_lines='skip') # TRIP DETAIL


## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 
df = df[df['platedate'].isin(common_utrecht_leek_platedate[1001:])]
with ProgressBar():
    df_trip_utrecht_leek = df.compute()
    
df_trip_utrecht_leek['Numberplate_copy'] = df_trip_utrecht_leek['Numberplate']
df_trip_utrecht_leek['platedate_copy'] = df_trip_utrecht_leek['platedate']
# len(df_trip_utrecht_leek['platedate_copy'].unique())

[########################################] | 100% Completed |  1min 31.6s


In [7]:
len(df_trip_utrecht_leek)

820522

In [8]:
# Function to extract indi dataframes (for each numberplate)
def dataframe_unique_numberplates(df):
    dfs = {}
    for platedate in df['platedate'].unique():
        try:
            dfs[platedate] = df.assign(**{platedate: df['platedate'].eq(platedate).astype(int)}).drop(columns='platedate')
        except:
            pass
        
    return dfs

df_trip_utrecht_leek=dataframe_unique_numberplates(df_trip_utrecht_leek)
# len(df_trip_utrecht_leek)
# df_trip_utrecht_leek.keys()
df_trip_utrecht_leek_keys=df_trip_utrecht_leek.keys()
# # df_trip_utrecht_leek_keys
df_trip_utrecht_leek_keys_list = list(df_trip_utrecht_leek.keys())
# print((df_trip_utrecht_leek_keys_list[0]))
# df_trip_utrecht_leek_keys

In [ ]:
# Filter trips between start and end location (right now we include trips from Location1 to Location2 and back).
# If you only want trips one way just comment the else statement

def extract_trip_between_start_end_location(df,start_location,end_location):
    df.drop(['index'],axis=1,inplace=True)
    df.reset_index(inplace=True)
    
    start_location_first_index=(df[df['Admin_area']==start_location].index)[0]
    end_location_first_index=(df[df['Admin_area']==end_location].index)[0]
    
    if start_location_first_index>end_location_first_index:
        df=df[end_location_first_index:start_location_first_index]
        return df
        
    else:
        df=df[start_location_first_index:end_location_first_index]
        return df

In [ ]:
# @TODO ->  replace this key with a list of keys and run a for loop and save dfs dynamically

def extracting_df_per_numberplate(df,key):
    
    df=df.get(key)
    # df=df.get(df_trip_utrecht_leek_keys_list[0])
    df = df.drop(df[df[key]==0].index)
    return df
# df_utrecht_leek_1=extracting_df_per_numberplate(df_trip_utrecht_leek,df_trip_utrecht_leek_keys_list[0])

In [ ]:
## saving the dataframes as a list.
list_of_unique_dataframes=list()
for i in df_trip_utrecht_leek_keys_list:
    try:
        unique_dfs=extracting_df_per_numberplate(df_trip_utrecht_leek,i)
        unique_dfs_trip=extract_trip_between_start_end_location(unique_dfs,'Utrecht','Leek') #location1,location2
        list_of_unique_dataframes.append(unique_dfs_trip)
    except:
        pass
# list_of_unique_dataframes

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df = df.drop(df[df['Numberplate']=='BTHN95'].index)
df = df.drop(df[df['Numberplate']=='BVFP92'].index)
df = df.drop(df[df['Numberplate']=='BTDS93'].index)
df = df.drop(df[df['Numberplate']=='BTFX65'].index)
df = df.drop(df[df['Numberplate']=='BTFX64'].index)
df = df.drop(df[df['Numberplate']=='BTFT45'].index)

df=df.loc[:,~df.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df=df.loc[:,~df.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
platedate_copy_list_utrecht_leek=df['platedate_copy']

In [ ]:
df_utrecht_leek_filtered = pd.concat(list_of_unique_dataframes) #concatenating list to form dataframe

df_utrecht_leek_filtered=df_utrecht_leek_filtered.loc[:,~df_utrecht_leek_filtered.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df_utrecht_leek_filtered=df_utrecht_leek_filtered.loc[:,~df_utrecht_leek_filtered.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
df_utrecht_leek_filtered

In [ ]:
platedate_copy_list_utrecht_leek=df['platedate_copy']
df_utrecht_leek_filtered = df_utrecht_leek_filtered[df_utrecht_leek_filtered['platedate_copy'].isin(platedate_copy_list_utrecht_leek)]

In [ ]:
df_utrecht_leek_filtered.to_csv(r"D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1\AOS_Summary\aos_summary_utrecht_leek_140M_road3.csv")

In [ ]:
fig = px.scatter_mapbox(df_utrecht_leek_filtered,lat="Latitude",lon="Longitude",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

Extracting platedates that pass through the same road and plugging them into complete dataframe to extract trips

In [ ]:
df_utrecht_leek_filtered.columns

In [ ]:
#grouping df based on platedate_copy 
# (Note:Here you need to manually check the range of distance that's common and add that to next step so that we can extract the same road)

df_utrecht_leek_filtered_grouped=df_utrecht_leek_filtered.groupby('platedate_copy').sum() 
df_utrecht_leek_filtered_grouped

rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 150) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=180)] #Road 1 - (Utrecht to Leek - Highway A27-A6-A7)
# rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 180) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=220)] #Road 2 - (Utrecht to Leek - Highway A28)

rslt_df=rslt_df.reset_index()
platedate_copy_same_road=rslt_df['platedate_copy']
platedate_copy_same_road # List of platedates that pass through the same road

In [ ]:
df = df_utrecht_leek_filtered[df_utrecht_leek_filtered['platedate_copy'].isin(platedate_copy_same_road)] # Extract complete df
df

In [ ]:
#grouping df based on platedate_copy 
# (Note:Here you need to manually check the range of distance that's common and add that to next step so that we can extract the same road)

df_utrecht_leek_filtered_grouped=df_utrecht_leek_filtered.groupby('platedate_copy').sum() 
df_utrecht_leek_filtered_grouped

rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 150) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=180)] #Road 1 - (Utrecht to Leek - Highway A27-A6-A7)
# rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 180) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=220)] #Road 2 - (Utrecht to Leek - Highway A28)

rslt_df=rslt_df.reset_index()
platedate_copy_same_road=rslt_df['platedate_copy']
platedate_copy_same_road # List of platedates that pass through the same road

### Processing motorway data

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df

### Utrecht-Leek

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df = df.drop(df[df['Numberplate']=='BTHN95'].index)
df = df.drop(df[df['Numberplate']=='BVFP92'].index)
df = df.drop(df[df['Numberplate']=='BTDS93'].index)
df = df.drop(df[df['Numberplate']=='BTFX65'].index)
df = df.drop(df[df['Numberplate']=='BTFX64'].index)
df = df.drop(df[df['Numberplate']=='BTFT45'].index)

df=df.loc[:,~df.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df=df.loc[:,~df.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
# platedate_copy_list_utrecht_leek=df['platedate_copy']

### Rotterdam-Antwerp

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\rotterdam_antwerp'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)

df = df.drop(df[df['Numberplate']=='BTDX76'].index)
df = df.drop(df[df['Numberplate']=='BSNZ14'].index)
df = df.drop(df[df['Numberplate']=='BSBT59'].index)
df = df.drop(df[df['Numberplate']=='BRVS98'].index)
df = df.drop(df[df['Numberplate']=='BSDP63'].index)
df = df.drop(df[df['Numberplate']=='BRXD53'].index)
df = df.drop(df[df['Numberplate']=='BSLR46'].index)
df = df.drop(df[df['Numberplate']=='BNXJ13'].index)
df = df.drop(df[df['Numberplate']=='BSTX97'].index)
df = df.drop(df[df['Numberplate']=='BSVZ49'].index)
df = df.drop(df[df['Numberplate']=='BSBR37'].index)

df=df.loc[:,~df.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df=df.loc[:,~df.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
df['Numberplate'] = df['platedate_copy'].str[0:6]
df['platedate'] = df['platedate_copy']
# platedate_copy_list_utrecht_leek=df['platedate_copy']

In [ ]:
def pre_processing(df):
    # df = df.drop(df[df['Average_speed_fpp']==0].index) # Dropping rows where driver is still 
    df['Numberplate_copy'] = df['Numberplate'] # creating copy of numberplate
    # df['platedate_copy'] = df['platedate'] # creating copy of platedate
    return df

def creating_grps(df):
    """This function will split df into two groups to calculate mean speed 
    and total distance travelled by each numberplate in Urban areas within that city"""
    
    df_groupby_mean = df.groupby(['Numberplate']).mean()
    df_groupby_mean.reset_index(inplace=True)
    
    df_groupby_sum = df.groupby(['Numberplate']).sum()
    df_groupby_sum.reset_index(inplace=True)
    
    return df_groupby_mean,df_groupby_sum

def dropping_columns_mean(df):
    df=df.drop(['Unnamed: 0', 'index', 'Id', 'Header_Id',
             'Trip_Summary_Id','Status',
             'Meters_travelled', 'Time_elapsed','Average_speed_fpp',
             'Point_AOS_headway', 'Road_type', 'Road_class',
             'Road_form', 'Speed_restriction', 'TNO_Valid', 'TNO_Day',
             'TNO_Area_Valid', 'TNO_Time_elapsed', 'TNO_Average_speed',
             'TNO_Time-stamp_hour', 'TNO Trip Hour','Overspeeding_event',
             'KmTravelled'], axis = 1)
    return df 

def dropping_columns_sum(df):
    df=df.drop(['Unnamed: 0', 'index', 'Id', 'Header_Id',
             'Trip_Summary_Id', 'Latitude', 'Longitude', 'Status',
             'Meters_travelled', 'Time_elapsed', 'Average_speed_fpp',
             'Point_AOS_headway', 'Road_type', 'Road_class',
             'Road_form', 'Speed_restriction', 'TNO_Valid', 'TNO_Day',
             'TNO_Area_Valid', 'TNO_Time_elapsed', 'TNO_Average_speed',
             'TNO_Time-stamp_hour', 'TNO Trip Hour','Point_speed'], axis = 1)
    return df

In [ ]:
df_intra_city_trip_detail = df.loc[df['Road_type'] == 1]
df_intra_city_trip_detail = (df_intra_city_trip_detail.pipe(cast_to_correct_dtype).pipe(rename_some_stuff).pipe(detect_overspeeding_count).pipe(some_processing))
df_intra_city_trip_detail['Numberplate'] = df_intra_city_trip_detail['platedate_copy'].str[0:6]

In [ ]:
df_intra_city_trip_detail

In [ ]:
df_groupby_mean,df_groupby_sum = (df_intra_city_trip_detail.pipe(pre_processing).pipe(creating_grps))
df_groupby_sum=dropping_columns_sum(df_groupby_sum)
df_groupby_mean=dropping_columns_mean(df_groupby_mean)


def merge(df_groupby_sum,df_groupby_mean):
    df = pd.merge(df_groupby_mean,df_groupby_sum, on='Numberplate', how='outer') #here Numberplate is common column
    # df=df.rename(columns={'Point_speed': 'Point_speed_mean_intra_city',
    #                'Overspeeding_event': 'Overspeeding_event_sum_intra_city',
    #                'KmTravelled': 'KmTravelled_sum_intra_city'})
    # df['norm_overspeeding']=df['Overspeeding_event_sum_intra_city']/df['KmTravelled_sum_intra_city']
    return df
    
df_city_grouped_trip_detail=merge(df_groupby_sum,df_groupby_mean)
df_city_grouped_trip_detail

In [ ]:
### Only considering drivers over 50 kms and removing outliers with 2nd condition
df_city_grouped_trip_detail=df_city_grouped_trip_detail[df_city_grouped_trip_detail['KmTravelled']>50]
df_city_grouped_trip_detail = df_city_grouped_trip_detail.drop(df_city_grouped_trip_detail[df_city_grouped_trip_detail['Point_speed']<20].index)

#### AOS Summary

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1\AOS_Summary'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_city_aos = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_city_aos_urban=df_city_aos[df_city_aos['Road_type_Rename']=='Motorway']
df_city_aos_urban['Numberplate'] = df_city_aos_urban['platedate_copy'].str[0:6]

In [ ]:
city_name="highwway_utrect_leek"

In [ ]:
%%capture
def total_aos_event_count(df):
    df=df.groupby(['Numberplate','Event type Rename']).size().unstack(fill_value=0)
    df.reset_index(inplace=True)
    return df

def merge_aos_events_trip_detail(total_aos_event_count,df_city_grouped_trip_detail):
    df = pd.merge(df_city_grouped_trip_detail, total_aos_event_count, on='Numberplate', how='outer') #here id is common column
    df = df.dropna(axis = 0, how = 'any')
    return df

def normalizing_events_count(df):
    df['norm_Indicators = OFF']=df[' Indicators = OFF']/df['KmTravelled']
    df['norm_brakes = OFF']=df['Brakes = OFF']/df['KmTravelled']
    df['norm_brakes = ON']=df['Brakes = ON']/df['KmTravelled']
    df['norm_headway_warning(long)']=df['Headway Warning (long)']/df['KmTravelled']
    df['norm_headway_warning(medium)']=df['Headway Warning (medium)']/df['KmTravelled']
    df['norm_headway_warning(short)']=df['Headway Warning (short)']/df['KmTravelled']
    df['norm_Headway Warning = OFF']=df['Headway Warning = OFF']/df['KmTravelled']
    df['norm_Lane Departure Warning = OFF']=df['Lane Departure Warning = OFF']/df['KmTravelled']
    df['norm_Left Indicator = ON']=df['Left Indicator = ON']/df['KmTravelled']
    df['norm_Left Lane Departure Warning = ON']=df['Left Lane Departure Warning = ON']/df['KmTravelled']
    # df['norm_Left and Right Indicator = ON']=df['Left and Right Indicator = ON']/df['KmTravelled']
    df['norm_Right Indicator = ON']=df['Right Indicator = ON']/df['KmTravelled']
    df['norm_Right Lane Departure Warning = ON']=df['Right Lane Departure Warning = ON']/df['KmTravelled']
    
    return df

df_total_aos_event_count=total_aos_event_count(df_city_aos_urban)
df_grp_aos_trip_detail=merge_aos_events_trip_detail(df_total_aos_event_count,df_city_grouped_trip_detail)
df_grp_aos_trip_detail=normalizing_events_count(df_grp_aos_trip_detail)
df_grp_aos_trip_detail['City'] = city_name


In [ ]:
numberplates_utrecht_leek_list = list(df_grp_aos_trip_detail['Numberplate'])

In [ ]:
df_grp_aos_trip_detail.to_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\df_grp_aos_trip_detail_utrecht_leek_highway.csv")

In [ ]:
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar



# # area_list = [location1,location2] # We can change these values to any start and end location
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape',usecols=["Numberplate","Latitude","Longitude","Point_speed","Meters_travelled","Road_type"])
# df=df[df['Road_type']==1]
# df = df[df['Numberplate'].isin(numberplates_utrecht_leek_list)]
# with ProgressBar():
#     df = df.compute()
    
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df_inter_city_trip_detail=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\TripDetail\utrecht_leek_trip_detail_final.csv")

In [ ]:

from shapely.geometry import Point, Polygon

# Create a Polygon
# coords_rotte = [(51.977901, 4.467480), (51.918861, 4.417012), (51.874370, 4.513485), (51.930857, 4.570123)]

coords_nl_belgium_de = [(51.319810, 3.199518), (51.141676, 4.044834), (51.151092, 4.433142), (51.160870, 4.911878),(51.119796, 5.379190),(50.916492, 5.347477),(50.732720, 5.558885),(50.646984, 5.868460),(50.682498, 6.225990),(50.774220, 6.788565

),(51.090741, 6.993645),(51.366330, 7.097475),(51.706392, 7.281298),(52.086558, 7.481535),(52.401805, 7.644418),(52.863343, 7.731970),(53.276982, 7.739262),(53.781184, 7.597500),(53.531285, 5.602048),(53.350568, 4.873847),(53.126881, 4.628008),(52.855526, 4.454778),(52.527262, 4.287084),(52.171565, 4.004032),(51.879936, 3.720315),(51.601678, 3.373136),
                (51.388379, 3.063189)]
poly_nl_belgium_de = Polygon(coords_nl_belgium_de)
poly_nl_belgium_de

In [ ]:
# initialize list of lists
from shapely.geometry import Point, Polygon
coords_nl_belgium_de = [(51.319810, 3.199518), (51.141676, 4.044834), (51.151092, 4.433142), (51.160870, 4.911878),(51.119796, 5.379190),(50.916492, 5.347477),(50.732720, 5.558885),(50.646984, 5.868460),(50.682498, 6.225990),(50.774220, 6.788565

),(51.090741, 6.993645),(51.366330, 7.097475),(51.706392, 7.281298),(52.086558, 7.481535),(52.401805, 7.644418),(52.863343, 7.731970),(53.276982, 7.739262),(53.781184, 7.597500),(53.531285, 5.602048),(53.350568, 4.873847),(53.126881, 4.628008),(52.855526, 4.454778),(52.527262, 4.287084),(52.171565, 4.004032),(51.879936, 3.720315),(51.601678, 3.373136),
                (51.388379, 3.063189)]
# Create the pandas DataFrame
df_poly_nl_belgium_de = pd.DataFrame(coords_nl_belgium_de, columns=['Lat', 'Lon'])

In [ ]:
polygon = Polygon([tuple(x) for x in df_poly_nl_belgium_de[['Lat', 'Lon']].to_numpy()])
df_inter_city_trip_detail['Within_NL'] = df_inter_city_trip_detail.apply(lambda x: polygon.contains(Point(x['Latitude'], x['Longitude'])), axis=1)

In [ ]:
df_inter_city_trip_detail = df_inter_city_trip_detail.drop(df_inter_city_trip_detail[df_inter_city_trip_detail['Within_NL']==False].index)
df_inter_city_trip_detail

In [ ]:
df_inter_city_trip_detail=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\TripDetail\utrecht_leek_not_complete_td.csv")

In [ ]:

def km_trav(df):
    df['Km_travelled']=df['Meters_travelled']/1000
    return df

def creating_grps(df):
    """This function will split df into two groups to calculate mean speed 
    and total distance travelled by each numberplate in Urban areas within that city"""
    
    df_groupby_mean = df.groupby(['Numberplate']).mean()
    df_groupby_mean.reset_index(inplace=True)
    
    df_groupby_sum = df.groupby(['Numberplate']).sum()
    df_groupby_sum.reset_index(inplace=True)
    
    return df_groupby_mean,df_groupby_sum

def dropping_columns_mean(df):
    df=df.drop(['Meters_travelled','Km_travelled'], axis = 1)
    return df 

def dropping_columns_sum(df):
    df=df.drop(['Latitude', 'Longitude', 'Point_speed', 'Within_NL', 'Meters_travelled',], axis = 1)
    return df


In [ ]:
df_groupby_mean,df_groupby_sum = (df_inter_city_trip_detail.pipe(km_trav).pipe(creating_grps))
df_groupby_sum=dropping_columns_sum(df_groupby_sum)
# df_groupby_mean
df_groupby_mean=dropping_columns_mean(df_groupby_mean)


def merge(df_groupby_sum,df_groupby_mean):
    df = pd.merge(df_groupby_mean,df_groupby_sum, on='Numberplate', how='outer') #here Numberplate is common column
    return df
    
df_city_grouped_trip_detail=merge(df_groupby_sum,df_groupby_mean)
df_city_grouped_trip_detail.drop(['Unnamed: 0.1_x','Unnamed: 0_x','Road_type_x','Within_NL','Unnamed: 0.1_y','Unnamed: 0_y','Road_type_y'],inplace=True,axis=1)

In [ ]:
df_city_grouped_trip_detail

In [ ]:
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar



# # area_list = [location1,location2] # We can change these values to any start and end location
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', encoding='unicode_escape')
# df=df[df['Road_type']==1]
# df = df[df['Numberplate'].isin(numberplates_utrecht_leek_list)]
# with ProgressBar():
#     df = df.compute()
    
# # df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df_inter_city_aos_summary=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\AOS_Summary\utrecht_leek_aos_summary_final.csv")

In [ ]:
df_inter_city_aos_summary=rename_some_stuff(df_inter_city_aos_summary)
df_inter_city_aos_summary

In [ ]:
city_name="Utrecht_leek"

In [ ]:
%%capture
def total_aos_event_count(df):
    df=df.groupby(['Numberplate','Event type Rename']).size().unstack(fill_value=0)
    df.reset_index(inplace=True)
    return df

def merge_aos_events_trip_detail(total_aos_event_count,df_city_grouped_trip_detail):
    df = pd.merge(df_city_grouped_trip_detail, total_aos_event_count, on='Numberplate', how='outer') #here id is common column
    df = df.dropna(axis = 0, how = 'any')
    return df

def normalizing_events_count(df):
    df['norm_Indicators = OFF']=df[' Indicators = OFF']/df['Km_travelled']
    df['norm_brakes = OFF']=df['Brakes = OFF']/df['Km_travelled']
    df['norm_brakes = ON']=df['Brakes = ON']/df['Km_travelled']
    df['norm_headway_warning(long)']=df['Headway Warning (long)']/df['Km_travelled']
    df['norm_headway_warning(medium)']=df['Headway Warning (medium)']/df['Km_travelled']
    df['norm_headway_warning(short)']=df['Headway Warning (short)']/df['Km_travelled']
    df['norm_Headway Warning = OFF']=df['Headway Warning = OFF']/df['Km_travelled']
    df['norm_Lane Departure Warning = OFF']=df['Lane Departure Warning = OFF']/df['Km_travelled']
    df['norm_Left Indicator = ON']=df['Left Indicator = ON']/df['Km_travelled']
    df['norm_Left Lane Departure Warning = ON']=df['Left Lane Departure Warning = ON']/df['Km_travelled']
    df['norm_Left and Right Indicator = ON']=df['Left and Right Indicator = ON']/df['Km_travelled']
    df['norm_Right Indicator = ON']=df['Right Indicator = ON']/df['Km_travelled']
    df['norm_Right Lane Departure Warning = ON']=df['Right Lane Departure Warning = ON']/df['Km_travelled']
    
    return df

df_total_aos_event_count=total_aos_event_count(df_inter_city_aos_summary)
df_grp_aos_trip_detail_complete=merge_aos_events_trip_detail(df_total_aos_event_count,df_city_grouped_trip_detail)
df_grp_aos_trip_detail_complete=normalizing_events_count(df_grp_aos_trip_detail_complete)
df_grp_aos_trip_detail_complete['City'] = city_name+"_complete"


In [ ]:
df_grp_aos_trip_detail_complete

In [ ]:
df_grp_aos_trip_detail_complete.to_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\df_grp_aos_trip_detail_complete_utrecht_leek_highway.csv")

### Clustering

In [ ]:
df_grp_aos_trip_detail_amsterdam=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\df_grp_aos_trip_detail_utrecht_leek_highway.csv")
df_grp_aos_trip_detail_complete_amsterdam=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\df_grp_aos_trip_detail_complete_utrecht_leek_highway.csv")
city_name='utrecht_leek'

In [ ]:
df_grouped_cities=pd.concat([df_grp_aos_trip_detail_amsterdam, df_grp_aos_trip_detail_complete_amsterdam])
df_grouped_cities

In [ ]:
df_grouped_cities.columns

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
feature="norm_Right Lane Departure Warning = ON"

In [ ]:
def preprocess(df,feature):
    """Preprocess data for KMeans clustering"""
    
    data = np.array(df[feature])
    data=data.reshape(-1, 1)
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    
    return data

In [ ]:

def elbow_plot(data):
    """Create elbow plot from normalized data"""
    
    sse = {}
    
    for k in range(2,10):
        kmeans = KMeans(n_clusters=k, random_state=1)
        kmeans.fit(data)
        sse[k] = kmeans.inertia_
    
    plt.title('Elbow plot for K selection')
    plt.xlabel('k')
    plt.ylabel('SSE')
    sns.pointplot(x=list(sse.keys()),y=list(sse.values()))
    plt.show()

data = preprocess(df_grouped_cities,feature)
elbow_plot(data)

In [ ]:
# data = preprocess(df_grp_aos_trip_detail,"Point_speed_mean_intra_city")

def silhouette_coeff(data):
    """Checking silhouette score"""
    
    range_n_clusters = range(2,10)
    
    for n_clusters in range_n_clusters:
        clusterer = KMeans(n_clusters=n_clusters)
        preds = clusterer.fit_predict(data)
        centers = clusterer.cluster_centers_

        score = silhouette_score(data, preds)
        print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))
    
silhouette_coeff(data)

In [ ]:
# data = preprocess(df_grp_aos_trip_detail,"Point_speed_mean_intra_city")

def find_k(data,feature,increment=0, decrement=0):
    """Find the optimum k clusters"""
    
    data = preprocess(data,feature)
    sse = {}
    
    for k in range(2, 21):
        kmeans = KMeans(n_clusters=k, random_state=1)
        kmeans.fit(data)
        sse[k] = kmeans.inertia_
    
    kn = KneeLocator(x=list(sse.keys()), 
                 y=list(sse.values()), 
                 curve='convex', 
                 direction='decreasing')
    k = kn.knee + increment - decrement
    return k

In [ ]:
def run_kmeans(df,feature, increment=0, decrement=0):
    """Run KMeans clustering, including the preprocessing of the data
    and the automatic selection of the optimum k. 
    """
    
    data = preprocess(df,feature)
    k = find_k(df,feature, increment, decrement)
    print(k)
    kmeans = KMeans(n_clusters=k,random_state=1,init='k-means++')
    kmeans.fit(data)
    return df.assign(cluster=kmeans.labels_)

In [ ]:
# feature="norm_headway_warning(short)"
clusters = run_kmeans(df_grouped_cities,feature, decrement=2)
clusters

In [ ]:
clusters = clusters.drop(clusters[clusters['norm_headway_warning(long)']>8].index)

In [ ]:
# feature="norm_headway_warning(short)"
sns.kdeplot(data=clusters, x=feature, hue="cluster",fill=True, palette="crest",
   alpha=.5, linewidth=2)
plt.title(r"Distribution of Mean Point Speed in "+"\n" + city_name +" (Urban Roads) - Clusters")
plt.legend(title='Clusters', labels=['0', '1','2', '3','4'])
plt.xlabel("Mean Point Speed (km/h)")

In [ ]:
clusters

In [ ]:
df_city=clusters.loc[clusters['City'] == "highwway_utrect_leek"]
df_not_city=clusters.loc[clusters['City'] == "Utrecht_leek_complete"]

In [ ]:
### Only considering drivers over 50 kms and removing outliers with 2nd condition
df_not_city=df_not_city[df_not_city['Km_travelled']>50]
df_not_city = df_not_city.drop(df_not_city[df_not_city['Point_speed']<20].index)

In [ ]:
df_outer = pd.merge(df_city, df_not_city, on='Numberplate', how='outer') #here Numberplate is common column
df_outer

In [ ]:
df_outer['cluster_comp'] = np.where(df_outer['cluster_x']==df_outer['cluster_y'], 'True', 'False')

In [ ]:
a=df_outer[df_outer['cluster_comp']=='True']
# a['clusters_utrecht'].value_counts()
len(df_outer[df_outer['cluster_comp']=='True'])/len(df_outer)*100

Viz

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df = df.drop(df[df['Numberplate']=='BTHN95'].index)
df = df.drop(df[df['Numberplate']=='BVFP92'].index)
df = df.drop(df[df['Numberplate']=='BTDS93'].index)
df = df.drop(df[df['Numberplate']=='BTFX65'].index)
df = df.drop(df[df['Numberplate']=='BTFX64'].index)
df = df.drop(df[df['Numberplate']=='BTFT45'].index)
df = df.drop(df[df['Numberplate']=='BTDB40'].index)
df = df.drop(df[df['Numberplate']=='BVFP91'].index)

df=df.loc[:,~df.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df=df.loc[:,~df.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
platedate_copy_list_utrecht_leek=df['platedate_copy']
df['Numberplate'] = df['platedate_copy'].str[0:6]

In [ ]:
df.columns

In [ ]:
fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",color="Numberplate",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1\AOS_Summary'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_city_aos = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_city_aos_urban=df_city_aos[df_city_aos['Road_type_Rename']=='Motorway']
df_city_aos_urban['Numberplate'] = df_city_aos_urban['platedate_copy'].str[0:6]
df_city_aos_urban

In [ ]:
df_city_aos_urban.columns

In [ ]:
df_city_aos_urban = df_city_aos_urban.drop(df_city_aos_urban[df_city_aos_urban['Numberplate']=="BTLV82"].index)


In [ ]:
fig = px.scatter_mapbox(df_city_aos_urban,lat="Latitude",lon="Longitude",color="Event type Rename",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\Rotterdam_Groningen\updated_dataset'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)

# df = df.drop(df[df['Numberplate']=='BTDX76'].index)
# df = df.drop(df[df['Numberplate']=='BSNZ14'].index)
# df = df.drop(df[df['Numberplate']=='BSBT59'].index)
# df = df.drop(df[df['Numberplate']=='BRVS98'].index)
# df = df.drop(df[df['Numberplate']=='BSDP63'].index)
# df = df.drop(df[df['Numberplate']=='BRXD53'].index)
# df = df.drop(df[df['Numberplate']=='BSLR46'].index)
# df = df.drop(df[df['Numberplate']=='BNXJ13'].index)
# df = df.drop(df[df['Numberplate']=='BSTX97'].index)
# df = df.drop(df[df['Numberplate']=='BSVZ49'].index)
# df = df.drop(df[df['Numberplate']=='BSBR37'].index)

df=df.loc[:,~df.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df=df.loc[:,~df.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
# df['Numberplate'] = df['platedate'].str[0:6]
# df['platedate'] = df['platedate_copy']
# platedate_copy_list_utrecht_leek=df['platedate_copy']

In [ ]:
fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
df.columns

In [ ]:
#grouping df based on platedate_copy 
# (Note:Here you need to manually check the range of distance that's common and add that to next step so that we can extract the same road)

df_sum=df.groupby('platedate_copy').sum() 
df_sum

rslt_df = df_sum.loc[(df_sum['KmTravelled'] >= 96) & (df_sum['KmTravelled'] <=100)] #Road 1 - (Utrecht to Leek - Highway A27-A6-A7)
# # # rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 180) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=220)] #Road 2 - (Utrecht to Leek - Highway A28)

rslt_df=rslt_df.reset_index()
# rslt_df
platedate_copy_same_road=rslt_df['platedate_copy']
platedate_copy_same_road # List of platedates that pass through the same road

In [ ]:
df = df[df['platedate_copy'].isin(platedate_copy_same_road)]
df

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\rotterdam_antwerp\aos_summary'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_aos_urban = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# df_aos_urban = df_aos_urban.drop(df_aos_urban[df_aos_urban['Numberplate']=='BSPD63'].index)
# df_aos_urban = df_aos_urban.drop(df_aos_urban[df_aos_urban['Numberplate']=='BRZX35'].index)
# df_aos_urban = df_aos_urban.drop(df_aos_urban[df_aos_urban['Numberplate']=='BRDD42'].index)
# df = df.drop(df[df['Numberplate']=='BSBT59'].index)
# df = df.drop(df[df['Numberplate']=='BRVS98'].index)
# df = df.drop(df[df['Numberplate']=='BSDP63'].index)
# df = df.drop(df[df['Numberplate']=='BRXD53'].index)
# df = df.drop(df[df['Numberplate']=='BSLR46'].index)
# df = df.drop(df[df['Numberplate']=='BNXJ13'].index)
# df = df.drop(df[df['Numberplate']=='BSTX97'].index)
# df = df.drop(df[df['Numberplate']=='BSVZ49'].index)
# df = df.drop(df[df['Numberplate']=='BSBR37'].index)

# df=df.loc[:,~df.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
# df=df.loc[:,~df.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
df_aos_urban['Numberplate'] = df_aos_urban['platedate'].str[0:6]
# df_aos_urban['platedate'] = df_aos_urban['platedate_copy']
# platedate_copy_list_utrecht_leek=df['platedate_copy']

In [ ]:
fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",color="Numberplate",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
numberplate_copy_list_rotte_ant=df_aos_urban['Numberplate']

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# area_list=numberplate_copy_list_rotte_ant

# area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',encoding='unicode_escape',delimiter=";")
df = df[df['Road_type']==1]
df = df[df['Numberplate'].isin(numberplate_copy_list_rotte_ant)]
with ProgressBar():
    df = df.compute()
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\TripDetail\rotte_antwerp_trip_detail_complete.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
platedate_copy_same_road

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

area_list=platedate_copy_same_road

# area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_80M.csv',encoding='unicode_escape',assume_missing=True)
df = df[df['Road_type']==1]
df = df[df['platedate'].isin(area_list)]
with ProgressBar():
    df = df.compute()
df.to_csv(r"D:\AOS FOT\Octo\pickle_files\rotterdam_antwerp\aos_summary_rotte_antwerp_80M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

area_list=platedate_copy_same_road

# area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_120M.csv',encoding='unicode_escape')
df = df[df['Road_type']==1]
df = df[df['platedate'].isin(area_list)]
with ProgressBar():
    df = df.compute()
df.to_csv(r"D:\AOS FOT\Octo\pickle_files\rotterdam_antwerp\aos_summary_rotte_antwerp_120M.csv")

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

area_list=platedate_copy_same_road

# area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_100M.csv',encoding='unicode_escape',assume_missing=True)
df = df[df['Road_type']==1]
df = df[df['platedate'].isin(area_list)]
with ProgressBar():
    df = df.compute()
df.to_csv(r"D:\AOS FOT\Octo\pickle_files\rotterdam_antwerp\aos_summary_rotte_antwerp_100M.csv")

In [30]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_motorway_td = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)

In [31]:
df_motorway_td=df_motorway_td.loc[:,~df_motorway_td.columns.str.startswith('B')]
df_motorway_td=df_motorway_td.loc[:,~df_motorway_td.columns.str.startswith('N')]

In [32]:
df_motorway_td.columns

Index(['index', 'Unnamed: 0', 'Id', 'Header_Id', 'Trip_Summary_Id',
       'Point_date-stamp', 'Point_time-stamp', 'Latitude', 'Longitude',
       'Status', 'Meters_travelled', 'Time_elapsed', 'Point_speed',
       'Average_speed_fpp', 'Point_AOS_headway', 'Road_type', 'Road_class',
       'Road_form', 'Speed_restriction', 'Admin_area', 'TNO_Time-stamp',
       'TNO_Valid', 'TNO_Day', 'TNO_Area_Valid', 'TNO_Time_elapsed',
       'TNO_Average_speed', 'TNO_Time-stamp_hour', 'TNO Trip Date',
       'TNO Trip Time', 'TNO Trip Hour', 'Road_type_Rename',
       'Overspeeding_event', 'KmTravelled', 'Part of Day', 'TNO_date_str',
       'platedate_copy'],
      dtype='object')

In [25]:
groupby_df_motorway_td = df_motorway_td.groupby("platedate_copy")["KmTravelled"].sum()

In [26]:
groupby_df_motorway_td

platedate_copy
BJSP232009-01-21    194.576
BLBF832009-05-11    195.632
BNLZ292009-01-06    193.001
BNXD692008-12-18    180.492
BPRR362008-12-13    199.260
                     ...   
BVFP922008-10-07    183.204
BVFP932008-11-22    180.354
NNB2882009-04-07    121.493
NNB2982009-04-09    120.449
NNB3022009-03-04    180.454
Name: KmTravelled, Length: 87, dtype: float64

In [16]:
#grouping df based on platedate_copy 
# (Note:Here you need to manually check the range of distance that's common and add that to next step so that we can extract the same road)

df_motorway_td_grouped=df_motorway_td.groupby('platedate_copy').sum() 
df_motorway_td_grouped

rslt_df = df_motorway_td_grouped.loc[(df_motorway_td_grouped['KmTravelled'] >= 60) & (df_motorway_td_grouped['KmTravelled'] <=90)] #Road 1 - (Utrecht-Eindhoven)


# rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 150) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=180)] #Road 1 - (Utrecht to Leek - Highway A27-A6-A7)
# rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 180) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=220)] #Road 2 - (Utrecht to Leek - Highway A28)

rslt_df=rslt_df.reset_index()
platedate_copy_same_road=rslt_df['platedate_copy']
platedate_copy_same_road # List of platedates that pass through the same road

0       BGTB712008-11-21
1       BLFR752009-04-01
2       BLLF652008-10-30
3       BLLF652009-01-22
4       BLLG102009-01-05
             ...        
789     BVSG722009-06-26
790     BVSR092009-03-29
791     BVTF352009-05-29
792     BVVV282009-06-05
793    WPR23X42009-02-01
Name: platedate_copy, Length: 794, dtype: object

In [17]:
df = df_motorway_td[df_motorway_td['platedate_copy'].isin(platedate_copy_same_road)] # Extract complete df
df

,index,Unnamed: 0,Id,Header_Id,Trip_Summary_Id,Point_date-stamp,Point_time-stamp,Latitude,Longitude,Status,...,TNO Trip Date,TNO Trip Time,TNO Trip Hour,Road_type_Rename,Overspeeding_event,KmTravelled,Part of Day,TNO_date_str,platedate_copy,WPR23X42009-02-01
4050,75231,12956330.0,112956328.0,247.0,247.0,2008-11-21 00:00:00.000,2009-06-09 08:55:54.000,52.140547,5.004359,1.0,...,2008-11-21,08:55:54,8.0,Urban,1.0,0.00,Early Morning,2008-11-21,BGTB712008-11-21,NaN
4051,75232,12956331.0,112956329.0,247.0,247.0,2008-11-21 00:00:00.000,2009-06-09 08:55:55.000,52.140406,5.004456,1.0,...,2008-11-21,08:55:55,8.0,Urban,1.0,0.00,Early Morning,2008-11-21,BGTB712008-11-21,NaN
4052,75233,12956332.0,112956330.0,247.0,247.0,2008-11-21 00:00:00.000,2009-06-09 08:55:56.000,52.140273,5.004547,1.0,...,2008-11-21,08:55:56,8.0,Urban,1.0,0.00,Early Morning,2008-11-21,BGTB712008-11-21,NaN
4053,75234,12956333.0,112956331.0,247.0,247.0,2008-11-21 00:00:00.000,2009-06-09 08:55:58.000,52.139998,5.004737,1.0,...,2008-11-21,08:55:58,8.0,Urban,1.0,0.00,Early Morning,2008-11-21,BGTB712008-11-21,NaN
4054,75235,12956334.0,112956332.0,247.0,247.0,2008-11-21 00:00:00.000,2009-06-09 08:56:10.000,52.138221,5.005973,1.0,...,2008-11-21,08:56:10,8.0,Motorway,0.0,0.00,Early Morning,2008-11-21,BGTB712008-11-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657863,243577,6322073.0,46322071.0,48.0,48.0,2009-03-29 00:00:00.000,2009-06-01 20:58:55.000,51.503599,5.416528,1.0,...,2009-03-29,20:58:55,20.0,Motorway,0.0,2.08,Evening,2009-03-29,BVSR092009-03-29,NaN
657864,243578,6322074.0,46322072.0,48.0,48.0,2009-03-29 00:00:00.000,2009-06-01 20:58:58.000,51.503087,5.417169,1.0,...,2009-03-29,20:58:58,20.0,Motorway,0.0,0.00,Evening,2009-03-29,BVSR092009-03-29,NaN
657865,243579,6322075.0,46322073.0,48.0,48.0,2009-03-29 00:00:00.000,2009-06-01 20:58:58.000,51.503087,5.417169,1.0,...,2009-03-29,20:58:58,20.0,Motorway,0.0,0.00,Evening,2009-03-29,BVSR092009-03-29,NaN
657866,243580,6322076.0,46322074.0,48.0,48.0,2009-03-29 00:00:00.000,2009-06-01 21:00:07.000,51.491156,5.428761,1.0,...,2009-03-29,21:00:07,21.0,Motorway,0.0,0.00,Night,2009-03-29,BVSR092009-03-29,NaN


In [33]:
df_motorway_td['Numberplate'] = df_motorway_td['platedate_copy'].str[0:6]

In [34]:
fig = px.scatter_mapbox(df_motorway_td,lat="Latitude",lon="Longitude",title="Utrecht-Leek  111 Trips (71 Trucks)",color="Numberplate")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()